# Lexique français
Permet de créer son propre lexique de mots français en fichier Excel .xlsx. Les mots ajoutés peuvent provenir directement du dictionnaire présent dans le dossier courrant (dico.csv) ou d'un ajout personnalisé.

In [4]:
import time
import random
import logging
import pandas as pd
from openpyxl import load_workbook

In [16]:
logger = logging.getLogger()
logger.setLevel(logging.WARNING)

In [5]:
# Pandas dataframe of the dictionary dico.csv
df = pd.read_csv("dico.csv")
df = df.sort_values("Mot")
df = df.dropna()
df = df.reset_index(drop=True)

In [17]:
# Opening the excel file
excel_file = load_workbook("lexi.xlsx")
sheet = excel_file.active

# Writing of the first line: column names
sheet["A1"] = "Mot"
sheet["B1"] = "Definitions"
sheet["C1"] = "Timestamp"

## Fonctions
Ces fonctions agiront en passerelles entre les données contenues dans le dictionnaire `.csv` et le fichier `.xlsx`. Elles peremttront entre autres d'ajouter des mots dans le lexique, d'en rechercher ou encore d'en supprimer

In [7]:
# In order to reduce the risks associated with too long or potentially 
# infinite iterations of the lexicon
LIMIT_ITER = 10000

In [8]:
def first_empty():
    """ Ruturns the index of the first empty cell
    in column A """
    
    idx = 2
    while True:
        # Cell is empty
        if not isinstance(sheet[f'A{idx}'].value, str):
            return idx
        
        # Cell is full
        else:
            idx+=1
            if idx == LIMIT_ITER:
                logging.critical("Watchdog : too many iterations")
                break
            else:
                continue

In [33]:
def add_word(word):
    """ Addition of a word in the lexicon which is 
    present in the dictionary """
    
    word = word.capitalize()

    D = df.loc[df['Mot'] == word]['Définitions']

    # Word not found
    if len(D) == 0:
        logging.warning(f"'{word}' not in dictionnary")
        return None

    # Word found
    else:
        logging.info(f"'{word}' found in dictionnary at idx {df.loc[df['Mot'] == word].index[0]}")
        D = eval(D.values[0]) # -> <list>

        idx = 2 # Index vertical de la colonne

        # Loop to first empty cell in lexicon
        while idx <= first_empty():

            # Word already present in the lexicon
            if sheet[f"A{idx}"].value == word:
                logging.warning(f"'{word}' not added : already in lexicon at cell A{idx}")
                return None

            # Cell is empty
            if not isinstance(sheet[f"A{idx}"].value, (str, int, float)) :

                # All the definitions of the word contained in the list (D) are merged 
                # into a single string in order to be inserted into the sheet cell
                defs = []
                for idx_def, definition in enumerate(D):
                    defs.append(f"{idx_def+1}) {definition}")

                sheet[f"A{str(idx)}"] = word # Add word

                sheet[f"B{str(idx)}"] = "".join(defs) # Add definitions
                
                sheet[f"C{str(idx)}"] = str(int(time.time())) # Add timestamp

                logging.info(f"'{word}' added in lexicon at idx A{idx}")
                break
            
            else:
                idx+=1
                if idx == LIMIT_ITER: # watchdog
                    logging.critical("Watchdog : too many iterations")
                    break
    
    excel_file.save("lexi.xlsx") # Save file after modification
    logging.debug("xlsx file saved after adding a word")

In [26]:
def search(word):
    """ Search a word in the lexicon.
    If the word is found in the lexicon, the function returns in a tuple : 
    (the index, the word, its definitions, the timestamp of the addition).
    If no word was found, returns None """
    
    word = word.capitalize()

    idx = 2
    found = False
    while idx < first_empty():
        
        # Word found
        if sheet[f'A{idx}'].value == word:
            found = True
            logging.info(f"'{word}' found in lexicon at idx {idx}")
            return (idx, word, sheet[f'B{idx}'].value, sheet[f'C{idx}'].value)
        
        # Still searching
        else:
            idx+=1
    
    # End of itération : word not found
    if found == False:
        logging.warning(f"'{word}' not found in lexicon")
        return None

In [11]:
def delete(word):
    """ Delete a word from the lexicon.
    The delete_rows() function from openpyxl automatically 
    fills the void left by the deletion"""
    
    word = word.capitalize()

    result = search(word) # returns None if no word was found

    # Word not found
    if result == None:
        logging.warning(f"Word '{word}' not deleted : not in lexicon")
        return None
    
    # Word found
    else:
        idx = result[0]
        sheet.delete_rows(idx)
        logging.info(f"'{word}' deleted from the lexicon")
        excel_file.save("lexi.xlsx")
        logging.debug("xlsx file saved after deletion")

In [12]:
def insert(word, definition):
    """ Insert a word in lexicon that not figure in dictionary """
    
    word = word.capitalize()

    result = search(word) # returns None if no word was found

    # Word already present
    if result != None:
        idx = result[0]
        logging.warning(f"Word '{word}' not inserted : already in lexicon at idx {idx}")
        return None
    
    # Word isn't present
    else:
        row = first_empty() # Where to write

        definition_txt = ""

        # 'definition' is a list or a tuple with multiple definitions.
        # Numbering of definitions and merging in a string
        if isinstance(definition, (list, tuple)):
            for idx, d in enumerate(definition):
                definition_txt += f"{idx+1}) {d} "
                sheet[f'A{row}'] = word
                sheet[f'B{row}'] = definition_txt
                sheet[f'C{row}'] = int(time.time())
                
        # 'definition' is a string
        elif isinstance(definition, (str)):
            definition_txt = "1) " + definition
            sheet[f'A{row}'] = word
            sheet[f'B{row}'] = definition_txt

        # Wrong type for 'definition'
        else:
            logging.critical(f"Word '{word}' not inserted : definition must be str or strs in list/tuple")
            return None
        
        excel_file.save("lexi.xlsx")
        logging.debug("xlsx file saved after insertion")

In [13]:
def add_random_words(sample_lenght, seed=None):
    """Adds a given number of random words to the lexicon"""
    
    if seed == None:
        sample = df.sample(sample_lenght)
    else:
        sample = df.sample(sample_lenght, random_state=seed)

    for idx, word in enumerate(sample["Mot"]):
        add_word(word)
    
    logging.debug(f"{sample_lenght} words have been added in the lexicon")
    
    excel_file.save("lexi.xlsx")
    logging.debug("xlsx file saved after adding random sample")

## Quelques exemples

### Remplissage du lexique
Afin d'avoir de la matière pour les prochains exemples nous allons créer un échantillon de 50 mots choisis aléatoirement dans le dictionnaire `dico.csv` que nous allons insérer un par un dans le lexique grace à la fonction `add_random_words` créée plus haut. 

Par soucis de reproductibilité, cet échantillon aléatoire sera soumis à un seed d'une valeur de 1.

In [18]:
add_random_words(50, seed=1)

### Brève exploration du lexique
Les fonctions d'ajout insèrent :
- Les **mots** sur la colonne A
- Les **définitions** sur la colonne B
- Le **timestamp** de l'ajout sur la colonne C

Chaque nouvel ajout se fait à la ligne précédent le dernier

In [20]:
# Affichage de la ligne 4
print(sheet['A4'].value) # Mot
print(sheet['B4'].value) # Définitions
print(sheet['C4'].value) # Timestamp

Rematernalisassiez
1) Deuxième personne du pluriel de l'imparfait du subjonctif du verbe rematernaliser.
1683127581


In [22]:
# Affichage des 10 premiers mots du lexique
wrds = 10
idx = 2

while idx < wrds+2:
    print(sheet[f'A{idx}'].value)
    idx+=1

Anglomanisâtes
Désestimeraient
Rematernalisassiez
Égrappaient
Reproportionnions
Réexfiltrera
Embabouina
Mincissais
Thermopathologique
Chier une pendule


### Utilisation des fonctions

#### search()
Recherche un mot dans le lexique, si le mot y est présent la fonction renvoie un `tuple` contenant :
- 0/ L'index de la ligne
- 1/ Le mot
- 2/ Les définitions
- 3/ Le timestamp

Si aucun mot n'a été trouvé, la fonction renvoie `None`

In [23]:
# Mot trouvé
search("Égrappaient")

(5,
 'Égrappaient',
 "1) Troisième personne du pluriel de l'indicatif imparfait du verbe égrapper.",
 '1683127581')

In [24]:
# Juste la définition
search("Égrappaient")[2]

"1) Troisième personne du pluriel de l'indicatif imparfait du verbe égrapper."

In [27]:
# Mot introuvable
search("Rompicher")

#### add_word()
Permet d'ajouter un mot dans le lexique si celui-ci est **présent dans le dictionnaire et absent du lexique**

In [28]:
add_word("Soupirail")

In [29]:
search("Soupirail")

(52,
 'Soupirail',
 "1) Ouverture pratiquée à la partie inférieure d'un édifice, pour donner un peu d'air et de jour à une cave, à un sous-sol, etc.2) :*  - La question est de savoir si c'est servir l'Europe, même libérale, que de la réintroduire par le soupirail du Parlement alors qu'elle [a] été expulsée par la porte du vote populaire. - (Le Monde diplomatique, 25 juin 2007)",
 '1683129181')

In [30]:
# Ajout d'un mot déjà présent dans le lexique
add_word("Égrappaient")

In [34]:
# Ajout d'un mot absent du dictionnaire
add_word("Rompicher")

#### insert()
Permet d'ajouter un mot dans le lexique qui n'est pas présent dans le dictionnaire. Pour cela il sera nécessaire de renseigner le mot ainsi que sa ou ses définitions.

Le paramètre `definition` peut être un `str` s'il n'y a qu'une definition pour le mot, ou un itérable de type `list` ou `tuple` s'il en possède plusieurs.

> **Note** : La fonction `insert()` appelle la fonction `search()` afin de vérifier si le mot à insérer n'est pas déjà présent dans le lexique, celle-ci emet un message Warning lorsqu'elle ne trouve pas le mot voulu, d'où l'affichage.

In [35]:
insert("Orange", ["Fruit", "Société française de télécommunications"])

In [36]:
search("Orange")

(53,
 'Orange',
 '1) Fruit 2) Société française de télécommunications ',
 1683129618)

#### delete()
Supprime la ligne correspondant à un mot donné en comblant le vide laissé par la suppression

In [37]:
# Affichage des 10 premiers mots du lexique
wrds = 10
idx = 2

while idx < wrds+2:
    print(sheet[f'A{idx}'].value)
    idx+=1

Anglomanisâtes
Désestimeraient
Rematernalisassiez
Égrappaient
Reproportionnions
Réexfiltrera
Embabouina
Mincissais
Thermopathologique
Chier une pendule


In [38]:
# Suppression du mot "Mincissais"
delete("Mincissais")

In [39]:
# Affichage des 10 premiers mots du lexique
# "Mincissais a été supprimé"
wrds = 10
idx = 2

while idx < wrds+2:
    print(sheet[f'A{idx}'].value)
    idx+=1

Anglomanisâtes
Désestimeraient
Rematernalisassiez
Égrappaient
Reproportionnions
Réexfiltrera
Embabouina
Thermopathologique
Chier une pendule
Enroquassions


#### first_empty()
Cette fonction retourne l'index de la ligne de la première cellule vide de la colonne A. Elle permet ainsi de déduire la longueur des données présentes dans le fichier .xlsx mais également de déterminer à quel index les nouveaux mots doivent être ajoutés sachant que ceux-ci s'ajoutent à la ligne suivant la dernière entrée du tableur

In [40]:
first_empty()

53